# 📊 Observabilidad y Linaje de Datos

Objetivo: implementar observabilidad end-to-end con logs estructurados, métricas (Prometheus), trazas (OpenTelemetry), linaje de datos (OpenLineage/DataHub) y SLOs.

- Duración: 120–150 min
- Dificultad: Alta
- Prerrequisitos: Senior 01 (Governance), pipelines en producción

## 1. Los tres pilares de observabilidad

- **Logs**: eventos estructurados (JSON) con contexto (trace_id, user, pipeline).
- **Metrics**: contadores, gauges, histogramas (latencia, throughput, errores).
- **Traces**: flujo de ejecución distribuido (OpenTelemetry).

## 2. Logs estructurados con loguru

In [ ]:
from loguru import logger
import sys, json
logger.remove()
logger.add(sys.stdout, serialize=True)  # JSON output

logger.info('pipeline_started', pipeline='ventas_etl', version='v1.2.3')
logger.error('validation_failed', pipeline='ventas_etl', error_count=5, sample_ids=[1,2,3])

## 3. Métricas con Prometheus (cliente Python)

In [ ]:
# from prometheus_client import Counter, Histogram, start_http_server
# import time, random
# 
# EVENTS_PROCESSED = Counter('events_processed_total', 'Total de eventos procesados', ['pipeline'])
# LATENCY = Histogram('processing_latency_seconds', 'Latencia de procesamiento', ['pipeline'])
# 
# start_http_server(8000)  # exponer métricas en :8000/metrics
# 
# while True:
#     with LATENCY.labels(pipeline='ventas_etl').time():
#         time.sleep(random.uniform(0.01, 0.1))
#         EVENTS_PROCESSED.labels(pipeline='ventas_etl').inc()
print('Ejemplo comentado; descomenta si tienes prometheus_client instalado')

## 4. Linaje de datos con OpenLineage

- Estándar abierto para rastrear de dónde vienen los datos y a dónde van.
- Integración con Airflow, Spark, dbt, Flink.
- Backend: Marquez, DataHub, Egeria.

In [ ]:
openlineage_example = r'''
# Integración con Airflow (plugin OpenLineage)
# airflow.cfg
[openlineage]
transport = {"type": "http", "url": "http://marquez:5000"}

# En cada DAG Run, Airflow emite eventos de linaje:
{
  "eventType": "START",
  "job": {"namespace": "data-platform", "name": "ventas_etl"},
  "inputs": [{"namespace": "s3", "name": "raw/ventas.csv"}],
  "outputs": [{"namespace": "s3", "name": "curated/ventas.parquet"}]
}
'''
print(openlineage_example.splitlines()[:15])

## 5. SLOs (Service Level Objectives)

- Definir objetivos medibles: ej. "99.9% de los pipelines completan < 30 min".
- Alertar si SLO en riesgo (error budget casi agotado).
- Revisar SLOs mensualmente y ajustar según necesidades del negocio.

In [ ]:
import pandas as pd
slo_example = pd.DataFrame([
  {'Pipeline':'ventas_etl', 'SLO':'Latencia p99 < 30min', 'Actual':'28min', 'Status':'✅'},
  {'Pipeline':'streaming_kafka', 'SLO':'Errores < 0.1%', 'Actual':'0.05%', 'Status':'✅'},
  {'Pipeline':'ml_training', 'SLO':'Disponibilidad > 99.5%', 'Actual':'99.2%', 'Status':'⚠️'},
])
slo_example

## 6. Dashboards y alertas

- Grafana para visualizar métricas de Prometheus.
- Alertmanager para notificar en Slack/PagerDuty.
- DataHub/Marquez UI para explorar linaje interactivo.
- Logs centralizados en ELK/Loki para troubleshooting.